In [1]:
import requests
import time

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator

Cargamos el texto de la nota de prensa del IPoM septiembre 2020 (transformado de pdf a txt [aquí](https://pdftotext.com/) y con un ajuste manual del espaciado de las frases) y seleccionamos las frases más largas para realizar el análisis de sentimiento.

In [11]:
text_file = open("np02092020.txt", "r", encoding="utf8")
lines = text_file.read().split('\n')

In [13]:
lines.sort(key=len, reverse=True)
nonEnglish_sentences = lines[0:28]

## Traducción Español - Inglés

Se sugiere que el texto sea traducido por un profesional, pero para este ejemplo usaremos la librería [deep-translator](https://github.com/nidhaloff/deep-translator) que disponibiliza la mayoría de los servicios de traducción gratuitos. Para este ejemplo usaremos el de Google Translate.

Ejemplo de uso:

```python
translated = GoogleTranslator(source='es', target='en').translate(text=text)
```

## Análisis de Sentimiento

Se sugiere usar librerías que puedan trabajar bien con texto traducido desde otro idioma. En particular hemos usado [vaderSentiment](https://github.com/cjhutto/vaderSentiment) con buenos resultados en el pasado. 

Para cada una de las frases se calculará un score compuesto (`compound`) que está normalizado entre -1 (negativo extremo) y +1 (positivo extremo). Es una métrica muy útil si se quiere tener una medida unidimensional del sentimiento de una sentencia dada.

Además puede servir para definir umbrales éstandarizados para clasificar frases como positivas, neutrales o negativas. Umbrales típicos son:

<ul>
<li>positivo: compound score >= 0.05</li>
<li>neutral: (compound score > -0.05) y (compound score < 0.05)</li>
<li>negativo: compound score <= -0.05</li>
</ul>
    
Los scores `pos`, `neu` y `neg` son las proporciones del texto que caen en cada categoría. Son las métricas más útiles si se quiere hacer un análisis multidimensional del sentimiento de una frase dada.

In [15]:
results = []
analyzer = SentimentIntensityAnalyzer()
for sentence in nonEnglish_sentences:
    try:
        translation = GoogleTranslator(source='es', target='en').translate(sentence)
        vs = analyzer.polarity_scores(translation)
        nl = '\n'
        print(f'{sentence}{nl}{str(vs)}{nl}')
        results.append([sentence, vs['compound'], vs['neg'], vs['neu'], vs['pos']])
        time.sleep(5)
    except:
        pass
    

Añade que revertir la pérdida de empleos, la caída de ingresos de los hogares, el deterioro del consumo y mantener un flujo de crédito acorde con las necesidades de capital de trabajo e inversión de las empresas, constituyen los mayores desafíos para la recuperación económica y para el aporte de la política pública en este proceso.
{'neg': 0.039, 'neu': 0.822, 'pos': 0.139, 'compound': 0.7003}

Estas tasas de crecimiento son coherentes con un escenario sanitario que permitirá el avance paulatino del desconfinamiento, donde los sectores económicos continuarán adaptando sus operaciones para desarrollar sus actividades y donde el crédito seguirá fluyendo para apoyar el proceso de recuperación.
{'neg': 0.0, 'neu': 0.799, 'pos': 0.201, 'compound': 0.8316}

El Banco Central de Chile publicó hoy el Informe de Política Monetaria (IPoM) de septiembre de 2020, en el que destaca que la evolución del escenario macroeconómico sigue determinada por el desarrollo del Covid-19 y las medidas de contenc

In [23]:
most_positive = max(results, key=lambda item: item[1])
most_negative = min(results, key=lambda item: item[1])

La frase más positiva de toda la nota de prensa:

In [24]:
most_positive[0]

'Estas tasas de crecimiento son coherentes con un escenario sanitario que permitirá el avance paulatino del desconfinamiento, donde los sectores económicos continuarán adaptando sus operaciones para desarrollar sus actividades y donde el crédito seguirá fluyendo para apoyar el proceso de recuperación.'

In [27]:
most_positive[1]

0.8316

La frase más negativa de toda la nota de prensa:

In [25]:
most_negative[0]

'Por el lado de la demanda agregada, el consumo y la inversión se han deprimido ante la pérdida de ingresos, las limitaciones a la movilidad y el contacto entre personas y la alta incertidumbre.'

In [28]:
most_negative[1]

-0.8176